<a href="https://colab.research.google.com/github/shanajdudekula/GEN-AI/blob/main/Genai11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio

In [ ]:
!ngrok config add-authtoken 33gIBhr6wU4dYniiWVpjGBqdEbt_3ZdmNGK2V4Rswr8K3bAYm


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import uvicorn
import asyncio


In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio

import nest_asyncio
import uvicorn
import asyncio
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
from pyngrok import ngrok
from datetime import datetime
import uuid

# Allow nested event loops
nest_asyncio.apply()

# Configure ngrok with your authtoken
NGROK_AUTHTOKEN = "33gIBhr6wU4dYniiWVpjGBqdEbt_3ZdmNGK2V4Rswr8K3bAYm"
ngrok.set_auth_token(NGROK_AUTHTOKEN)
print("✅ Ngrok authtoken configured successfully!")

# FastAPI app
app = FastAPI(
    title="Chat App",
    description="A simple real-time chat application",
    version="1.0.0"
)

# Enhanced message storage
chat_history = []

class Message(BaseModel):
    sender: str
    text: str
    message_id: Optional[str] = None
    timestamp: Optional[str] = None

@app.post("/chat", response_model=dict)
async def send_message(message: Message):
    """Send a new message to the chat"""
    try:
        # Add metadata
        message_dict = message.dict()
        message_dict["message_id"] = str(uuid.uuid4())
        message_dict["timestamp"] = datetime.now().isoformat()

        chat_history.append(message_dict)

        # Keep only last 100 messages to prevent memory issues
        if len(chat_history) > 100:
            chat_history.pop(0)

        return {
            "status": "Message received",
            "message_id": message_dict["message_id"],
            "timestamp": message_dict["timestamp"]
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error sending message: {str(e)}")

@app.get("/history", response_model=List[Message])
async def get_history(limit: Optional[int] = None):
    """Get chat history with optional limit"""
    try:
        if limit and limit > 0:
            return chat_history[-limit:]
        return chat_history
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error retrieving history: {str(e)}")

@app.delete("/clear")
async def clear_history():
    """Clear all chat history"""
    try:
        chat_history.clear()
        return {"status": "Chat history cleared"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error clearing history: {str(e)}")

@app.get("/")
async def root():
    return {
        "message": "Chat API is running!",
        "docs": "/docs",
        "endpoints": {
            "send_message": "POST /chat",
            "get_history": "GET /history",
            "clear_history": "DELETE /clear"
        }
    }

# Enhanced server setup
try:
    # Kill existing tunnels
    ngrok.kill()

    # Create tunnel with authenticated account
    public_url = ngrok.connect(8000, proto="http").public_url
    print(f"🚀 Public URL: {public_url}")
    print(f"📚 Swagger Docs: {public_url}/docs")
    print(f"🔗 Direct API: {public_url}/chat")
    print("\n💡 Try these commands:")
    print(f'curl -X POST "{public_url}/chat" -H "Content-Type: application/json" -d \'{{"sender": "User1", "text": "Hello World!"}}\'')
    print(f'curl "{public_url}/history"')
    print(f'curl -X DELETE "{public_url}/clear"')

except Exception as e:
    print(f"❌ Ngrok error: {e}")
    print("🔧 Trying to run without public URL...")

# Run FastAPI server
try:
    config = uvicorn.Config(
        app=app,
        host="0.0.0.0",
        port=8000,
        log_level="info",
        access_log=True
    )
    server = uvicorn.Server(config)

    print("✅ Server starting... Press Ctrl+C to stop")
    loop = asyncio.get_event_loop()
    loop.create_task(server.serve())

except Exception as e:
    print(f"❌ Server error: {e}")

✅ Ngrok authtoken configured successfully!


❌ Ngrok error: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: Your account is requesting a dev domain that does not exist. Visit https://dashboard.ngrok.com/domains to request a dev domain.\r\n\r\nERR_NGROK_15013\r\n"}}

🔧 Trying to run without public URL...
✅ Server starting... Press Ctrl+C to stop
